In [3]:
import heapq
class PuzzleNode:
    def __init__(self, state, parent=None, move=None, depth=0, cost=0):
        self.parent = parent
        self.state = state
        self.move = move
        self.depth = depth
        self.cost = cost
    def __lt__(self, other):
        return self.cost < other.cost
    def __eq__(self, other):
        return self.state == other.state
    def __hash__(self):
        return hash(str(self.state))
class NPuzzle:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
        self.size = len(goal_state)
        self.goal_pos = self.GoalPosition()
    def GoalPosition(self):
      return {val: (a, b) for a, r in enumerate(self.goal_state) for b, val in enumerate(r)}
    def heuristic(self, node):
      return sum(abs(a - self.goal_pos[val][0]) + abs(b - self.goal_pos[val][1])
               for a, r in enumerate(node.state)
               for b, val in enumerate(r)
               if val != 0)
    def neighbors(self, node):
        neigh = []
        for a, r in enumerate(node.state):
            for b, val in enumerate(r):
                if val == 0:
                    empty_a, empty_b = a, b
        moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        for move_a, move_b in moves:
            new_a, new_b = empty_a + move_a, empty_b + move_b
            if 0 <= new_a < self.size and 0 <= new_b < self.size:
                new_state = [r[:] for r in node.state]
                new_state[empty_a][empty_b], new_state[new_a][new_b] = new_state[new_a][new_b], new_state[empty_a][empty_b]
                neigh.append(PuzzleNode(new_state, parent=node, move=(move_a, move_b), depth=node.depth+1, cost=node.depth+1+self.heuristic(node)))
        return neigh
    def reconstruct_path(self, node):
        path = []
        while node.parent:
            path.append(node.move)
            node = node.parent
        return list(reversed(path))
    def astar_algo(self):
      OpenList = []
      ClosedList = set()
      initial_node = PuzzleNode(self.initial_state)
      heapq.heappush(OpenList, initial_node)
      while OpenList:
          current_node = heapq.heappop(OpenList)
          if current_node.state == self.goal_state:
              return self.reconstruct_path(current_node)
          ClosedList.add(current_node)
          neigh = self.neighbors(current_node)
          for n in neigh:
              if n in ClosedList:
                  continue
              if n not in OpenList:
                  heapq.heappush(OpenList, n)
              else:
                  existing_neighbor_index = next(i for i, item in enumerate(OpenList) if item == n)
                  existing_neighbor = OpenList[existing_neighbor_index]
                  if n.cost < existing_neighbor.cost:
                      existing_neighbor.cost = n.cost
                      existing_neighbor.parent = n.parent
                      existing_neighbor.move = n.move
      return None
initial_state = [[2, 8, 1],[0, 4, 3],[7, 6, 5]]
goal_state = [[1, 2, 3],[8, 0, 4],[7, 6, 5]]
solve = NPuzzle(initial_state, goal_state)
solution = solve.astar_algo()
print("Solution:", solution)

Solution: [(-1, 0), (0, 1), (0, 1), (1, 0), (0, -1), (0, -1), (-1, 0), (0, 1), (1, 0)]
